In [1]:
!pip install s3fs --quiet
!pip install zarr --quiet
!pip install openpyxl --quiet

In [7]:
import os
import fsspec
import pandas as pd
import xarray as xr
import dask
import numpy as np
from dask.distributed import Client
from dask.distributed import progress

In [3]:
client = Client(n_workers=12, memory_limit='2GB')

In [4]:
fs = fsspec.filesystem('s3', anon=True)
fs.glob('noaa-nwm-retrospective-2-1-zarr-pds/')

['noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/gwout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/index.html',
 'noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/ldasout.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/precip.zarr',
 'noaa-nwm-retrospective-2-1-zarr-pds/rtout.zarr']

In [5]:
file = fs.glob('noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr')
ds = xr.open_dataset(fs.get_mapper(file[0]), engine='zarr', backend_kwargs={'consolidated': True})

In [6]:
ds

<xarray.Dataset>
Dimensions:         (feature_id: 5783, time: 367439)
Coordinates:
  * feature_id      (feature_id) int32 491 531 747 ... 947070204 1021092845
    latitude        (feature_id) float32 ...
    longitude       (feature_id) float32 ...
  * time            (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T...
Data variables:
    crs             |S1 ...
    inflow          (time, feature_id) float64 ...
    outflow         (time, feature_id) float64 ...
    water_sfc_elev  (time, feature_id) float32 ...
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id

In [31]:
# Given geolocation
given_lat = 40.941
given_lon = -112.155

# Find the feature_id with the closest lat and lon
distances = np.sqrt((ds['latitude'] - given_lat)**2 + (ds['longitude'] - given_lon)**2)
closest_feature_id = int(distances.argmin().values)
closest_feature_id

2600

In [39]:
ds.isel(feature_id=1204).feature_id

<xarray.DataArray 'feature_id' ()>
array(4603518, dtype=int32)
Coordinates:
    feature_id  int32 4603518
    latitude    float32 41.58
    longitude   float32 -112.3
Attributes:
    cf_role:    timeseries_id
    comment:    ComID from NHDPlusV2 waterbody layer
    long_name:  Lake ComID

In [44]:
ds.sel(feature_id=946020001,method='nearest')

<xarray.Dataset>
Dimensions:         (time: 367439)
Coordinates:
    feature_id      int32 947070203
    latitude        float32 46.13
    longitude       float32 -119.0
  * time            (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T...
Data variables:
    crs             |S1 ...
    inflow          (time) float64 ...
    outflow         (time) float64 ...
    water_sfc_elev  (time) float32 ...
    distances       float32 7.84
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id